In [1]:
import nltk
import re
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import tensorflow as tf

import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, SimpleRNN, Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, MaxPooling1D, SimpleRNN, Dense, Dropout, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization,SpatialDropout1D
from keras.optimizers import RMSprop


from keras.utils import to_categorical
from keras.models import Model


In [2]:
dsk = pd.read_excel('/kaggle/input/kurdishenglish/KDFND_Anlyzed_Cleaned_Filtered_Labeld.xlsx')


In [3]:
# Remove rows with any null values
dsk = dsk.dropna(subset=['Text'])

In [4]:
dsk["Source"] = dsk["Source"].astype(str)
dsk["Text"] = dsk["Text"].astype(str)

#dsk["Article"] = dsk["Source"]+' '+ dsk["Text"]
dsk["Article"] = dsk["Text"]

dsk.sample(frac = 1) #Shuffle 100% Text

dsk.loc[dsk.label == 'Real', 'label'] = 0
dsk.loc[dsk.label == 'Fake', 'label'] = 1

dsk = dsk.loc[:,['Article', 'label']]
dsk = dsk.dropna()

In [5]:
dsk.isna().sum()

Article    0
label      0
dtype: int64

In [6]:
grouped_by_class = dsk.groupby('label').count()
grouped_by_class

,Article
label,
0,50751
1,50211


In [7]:
# Check for null/empty rows
print(dsk['Article'].isna().sum())  # Count NaN entries
print((dsk['Article'].str.strip() == '').sum())  # Count empty strings

# Remove rows where 'Article' is an empty string after stripping whitespace
dsk = dsk[dsk['Article'].str.strip() != '']

# Check for null/empty rows
print(dsk['Article'].isna().sum())  # Count NaN entries
print((dsk['Article'].str.strip() == '').sum())  # Count empty strings

0
0
0
0


In [8]:
# Assuming dsk is your DataFrame
# Separate the majority and minority classes
df_majority = dsk[dsk['label'] == 0]
df_minority = dsk[dsk['label'] == 1]

# Oversample the minority class
#df_minority_oversampled = df_minority.sample(len(df_majority), replace=True, random_state=2020)
df_majority_undersampled = df_majority.sample(len(df_minority), replace=True, random_state=2020)

# Combine the majority class with the oversampled minority class
#dsk_balanced = pd.concat([df_majority, df_minority_oversampled])
dsk_balanced = pd.concat([df_minority, df_majority_undersampled])

# Shuffle the dataset
dskb = dsk_balanced.sample(frac=1, random_state=2020).reset_index(drop=True)

# Check the new class distribution
grouped_by_class_balanced = dskb.groupby('label').count()
print(grouped_by_class_balanced)

       Article
label         
0        50211
1        50211


In [9]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(dskb['label']), y=dskb['label'])
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print("Class weights:", class_weights_dict)

Class weights: {0: 1.0, 1: 1.0}


In [10]:
def wordpre(text):
    if not isinstance(text, str):
        return ""  # Return empty string for non-string inputs
    # Normalize Kurdish characters
    #text = text.replace("ك", "ک").replace("ە", "ه")
    
    # Remove URLs, special characters, and numbers
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)      # Remove numbers
    
    # Remove extra spaces
    text = text.strip()
    
    return text
##  Applying the wordpre method to the dataset
dskb['Article']=dskb['Article'].apply(wordpre)

In [11]:
from collections import Counter
# Tokenize and count word frequencies
all_words = ' '.join(dskb).split()
word_freq = Counter(all_words)

# Identify potential stopwords (e.g., words appearing very frequently)
potential_stopwords = [word for word, freq in word_freq.items() if freq > 1]

# Example stopwords list (refined manually)
kurdish_stopwords = [
     "ئێمە","ئێوە","ئەم","ئەو"
     ,"ئەوان","ئەوەی","بۆ","بێ","بێجگە","بە","بەبێ","بەدەم","بەردەم","بەرلە","بەرەوی","بەرەوە","بەلای","بەپێی","تۆ","تێ","جگە","دوای","دوو","دە"
     ,"دەکات","دەگەڵ","سەر","لێ","لە","لەبابەت","لەباتی","لەبارەی","لەبرێتی","لەبن","لەبەر","لەبەینی","لەدەم","لەرێ","لەرێگا","لەرەوی","لەسەر","لەلایەن"
     ,"لەناو","لەنێو","لەو","لەپێناوی","لەژێر","لەگەڵ","من","ناو","نێوان","هەر","هەروەها","و","وەک","پاش","پێ","پێش","چەند","کرد","کە","ی"

] + potential_stopwords

kupunctuation = {'!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~'}

# Convert set to list before concatenation
ku_stopwords = kurdish_stopwords + list(kupunctuation)

def remove_stopwords(text, stop_words):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)
    
# Apply the function to each article
dskb['Article'] = dskb['Article'].apply(lambda text: remove_stopwords(text, ku_stopwords))

In [12]:
# Check for null/empty rows
#print(dskb['Article'].isna().sum())  # Count NaN entries
#print((dskb['Article'].str.strip() == '').sum())  # Count empty strings

# Remove rows where 'Article' is an empty string after stripping whitespace
#dskb = dskb[dskb['Article'].str.strip() != '']

# Check for null/empty rows
#print(dskb['Article'].isna().sum())  # Count NaN entries
#print((dskb['Article'].str.strip() == '').sum())  # Count empty strings

In [13]:
#dskb.duplicated().sum()

In [14]:
# Check for duplicates
#print(f"Number of duplicate rows: {dskb.duplicated().sum()}")


In [15]:
#print(dskb.duplicated().sum())
#dskb.drop_duplicates(inplace=True)
# Remove rows where 'Article' is an empty string after stripping whitespace
#dskb = dskb[dskb['Article'].str.strip() != '']
#print(dskb['Article'].isnull().sum())  # Count of missing values

# Handling missing values
#dskb['Article'] = dskb['Article'].fillna('')  # Replace NaNs with empty strings


In [16]:
glove_twitter = "/kaggle/input/glove-twitter/glove.twitter.27B.100d.txt"

In [17]:
# Load GloVe vectors
embedding_index = {}
with open(glove_twitter, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = vectors

print(f"Loaded {len(embedding_index)} word vectors.")

Loaded 1193514 word vectors.


In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dskb['Article'])
word_index = tokenizer.word_index
print(f"Vocabulary size: {len(word_index)}")

# Create embedding matrix
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print("Embedding matrix shape:", embedding_matrix.shape)

Vocabulary size: 184255
Embedding matrix shape: (184256, 100)


In [19]:
# Example GloVe embedding matrix
glove_embedding_matrix = np.random.rand(len(word_index)+1, 100)  # Random example

# L2 normalization of GloVe embeddings
l2_norm = np.linalg.norm(glove_embedding_matrix, axis=1, keepdims=True)

# Check for zero vectors and handle them
# Set zero vectors to a small value to avoid division by zero
l2_norm[l2_norm == 0] = 1e-10  # or any small constant

# Normalize the embeddings
normalized_glove_embeddings = embedding_matrix / l2_norm

# Verify the shape of the normalized embeddings
print("Normalized GloVe Embeddings Shape:", normalized_glove_embeddings.shape)

print("l2_norm:", l2_norm)

Normalized GloVe Embeddings Shape: (184256, 100)
l2_norm: [[5.67830824]
 [6.40248835]
 [6.02235761]
 ...
 [6.07573766]
 [5.89148818]
 [5.78081075]]


In [20]:

# Pad sequences
X = pad_sequences(tokenizer.texts_to_sequences(dskb['Article']), maxlen=100)
y = dskb['label']

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# Ensure labels are NumPy arrays and of the correct type
y_train = np.array(y_train, dtype=np.float32)
y_val = np.array(y_val, dtype=np.float32)


In [21]:
print("Input shape:", (X_train.shape[1], 1))

Input shape: (100, 1)


In [22]:
from keras.layers import GRU
model = Sequential([
    Embedding(input_dim=len(word_index) + 1, 
              output_dim=embedding_dim, 
              #input_length=100, 
              weights=[embedding_matrix], 
              trainable=True),  # Freeze pre-trained embeddings
    #LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.3, input_shape=(X_train.shape[1], 1)),
    #LSTM(32, dropout=0.3, recurrent_dropout=0.3),
    LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.3),
    GRU(50),
    Dropout(0.3),
    Dense(100, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │      18,425,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,425,600 (70.29 MB)

 Trainable params: 18,425,600 (70.29 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

# Train model
history = model.fit(X_train, y_train, epochs=2, batch_size=64, validation_data=(X_val, y_val),
    verbose=1, callbacks=[early_stopping, lr_scheduler])

# Evaluate model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy}")
print(f"Validation loss: {val_loss}")

Epoch 1/2
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 389s 306ms/step - accuracy: 0.7711 - loss: 0.4831 - val_accuracy: 0.8625 - val_loss: 0.3338 - learning_rate: 0.0010
Epoch 2/2
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 386s 307ms/step - accuracy: 0.9197 - loss: 0.2158 - val_accuracy: 0.8700 - val_loss: 0.3332 - learning_rate: 0.0010
628/628 ━━━━━━━━━━━━━━━━━━━━ 17s 28ms/step - accuracy: 0.8679 - loss: 0.3396
Validation Accuracy: 0.8700025081634521
Validation loss: 0.3332171142101288


In [25]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

# Train the model

history = model.fit(
    X_train, y_train, 
    epochs=3, 
    batch_size=64, 
    validation_split=0.2, 
    verbose=1,
    callbacks=[early_stopping, lr_scheduler]
)

from sklearn.metrics import accuracy_score, confusion_matrix
# Evaluate the model
#loss, accuracy = model.evaluate(X_val, y_val)
#print(f'Test Accuracy: {accuracy:.4f}')

# Predictions
y_pred = (model.predict(X_val) > 0.5).astype("int32")

# Metrics
print("Classification Report:")
print(classification_report(y_val, y_pred, zero_division=1))

accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy * 100:.4f}%")

Epoch 1/3
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 316s 315ms/step - accuracy: 0.9776 - loss: 0.0674 - val_accuracy: 0.9477 - val_loss: 0.1476 - learning_rate: 0.0010
Epoch 2/3
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 317s 315ms/step - accuracy: 0.9863 - loss: 0.0413 - val_accuracy: 0.9416 - val_loss: 0.1738 - learning_rate: 0.0010
Epoch 3/3
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 317s 315ms/step - accuracy: 0.9904 - loss: 0.0303 - val_accuracy: 0.9374 - val_loss: 0.2282 - learning_rate: 0.0010
628/628 ━━━━━━━━━━━━━━━━━━━━ 17s 28ms/step
Classification Report:
              precision    recall  f1-score   support

         0.0       0.87      0.87      0.87     10142
         1.0       0.87      0.86      0.87      9943

    accuracy                           0.87     20085
   macro avg       0.87      0.87      0.87     20085
weighted avg       0.87      0.87      0.87     20085

Accuracy: 86.7364%


In [35]:

# Define model
model = Sequential()
#model.add(Input(shape=()))
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=100, # Pad sequences to this length
                    trainable=False)) # Freeze embedding layer
model.add(LSTM(128, return_sequences=False, input_shape=(X_train.shape[1], 1)))  # 3D input   
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=1e-3), metrics=['accuracy'])
print(model.summary())

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_15 (Embedding)             │ ?                           │      18,435,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_15 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,435,200 (70.32 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 18,435,200 (70.32 MB)

None


In [36]:

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping, lr_scheduler])

# Evaluate model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy}")

Epoch 1/10
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 145s 142ms/step - accuracy: 0.8356 - loss: 0.3333 - val_accuracy: 0.8758 - val_loss: 0.2634 - learning_rate: 0.0010
Epoch 2/10
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 141s 141ms/step - accuracy: 0.8808 - loss: 0.2546 - val_accuracy: 0.8774 - val_loss: 0.2576 - learning_rate: 0.0010
Epoch 3/10
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 142s 141ms/step - accuracy: 0.8790 - loss: 0.2572 - val_accuracy: 0.8777 - val_loss: 0.2572 - learning_rate: 0.0010
Epoch 4/10
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 140s 140ms/step - accuracy: 0.8841 - loss: 0.2491 - val_accuracy: 0.8768 - val_loss: 0.2571 - learning_rate: 0.0010
Epoch 5/10
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 141s 140ms/step - accuracy: 0.8843 - loss: 0.2507 - val_accuracy: 0.8794 - val_loss: 0.2546 - learning_rate: 0.0010
Epoch 6/10
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 137s 136ms/step - accuracy: 0.8859 - loss: 0.2466 - val_accuracy: 0.8794 - val_loss: 0.2569 - learning_rate: 0.0010
Epoch 7/10
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 139s 138ms

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, MaxPooling1D, SimpleRNN, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization,SpatialDropout1D
from keras.optimizers import RMSprop

# Define model
model = Sequential()
#model.add(Input(shape=()))
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=100, # Pad sequences to this length
                    trainable=False)) # Freeze embedding layer
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], 1)))  # 3D input
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2)),
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=1e-3), metrics=['accuracy'])
print(model.summary())

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_16 (Embedding)             │ ?                           │      18,435,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_16 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_17 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_21 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,435,200 (70.32 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 18,435,200 (70.32 MB)

None


In [38]:

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping, lr_scheduler])

# Evaluate model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy}")

Epoch 1/5
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 115s 111ms/step - accuracy: 0.8306 - loss: 0.3633 - val_accuracy: 0.8750 - val_loss: 0.2626 - learning_rate: 0.0010
Epoch 2/5
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 110s 110ms/step - accuracy: 0.8804 - loss: 0.2582 - val_accuracy: 0.8753 - val_loss: 0.2609 - learning_rate: 0.0010
Epoch 3/5
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 111s 111ms/step - accuracy: 0.8825 - loss: 0.2538 - val_accuracy: 0.8788 - val_loss: 0.2567 - learning_rate: 0.0010
Epoch 4/5
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 111s 110ms/step - accuracy: 0.8825 - loss: 0.2531 - val_accuracy: 0.8780 - val_loss: 0.2563 - learning_rate: 0.0010
Epoch 5/5
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 110s 110ms/step - accuracy: 0.8829 - loss: 0.2499 - val_accuracy: 0.8792 - val_loss: 0.2574 - learning_rate: 0.0010
503/503 ━━━━━━━━━━━━━━━━━━━━ 14s 28ms/step - accuracy: 0.8764 - loss: 0.2611
Validation Accuracy: 0.8780184388160706


In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, MaxPooling1D, SimpleRNN, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization,SpatialDropout1D
from keras.optimizers import RMSprop

# Define model
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Embedding(input_dim=len(word_index) + 1,
                   output_dim=embedding_dim,weights=[embedding_matrix],
                    input_length=100,trainable=False)) # Freeze embedding layer
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2)),
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=1e-3), metrics=['accuracy'])
print(model.summary())

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_19 (Embedding)             │ (None, 100, 100)            │      18,435,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_20 (LSTM)                       │ (None, 64)                  │          42,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_24 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 16)                  │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,478,497 (70.49 MB)

 Trainable params: 43,297 (169.13 KB)

 Non-trainable params: 18,435,200 (70.32 MB)

None


In [43]:

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping, lr_scheduler])

# Evaluate model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy}")

Epoch 1/5
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 69s 67ms/step - accuracy: 0.8375 - loss: 0.3435 - val_accuracy: 0.8754 - val_loss: 0.2602 - learning_rate: 0.0010
Epoch 2/5
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 67s 67ms/step - accuracy: 0.8725 - loss: 0.2625 - val_accuracy: 0.8790 - val_loss: 0.2601 - learning_rate: 0.0010
Epoch 3/5
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 68s 68ms/step - accuracy: 0.8793 - loss: 0.2573 - val_accuracy: 0.8773 - val_loss: 0.2581 - learning_rate: 0.0010
Epoch 4/5
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 69s 69ms/step - accuracy: 0.8853 - loss: 0.2482 - val_accuracy: 0.8765 - val_loss: 0.2609 - learning_rate: 0.0010
Epoch 5/5
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 69s 68ms/step - accuracy: 0.8846 - loss: 0.2495 - val_accuracy: 0.8814 - val_loss: 0.2553 - learning_rate: 0.0010
503/503 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.8808 - loss: 0.2582
Validation Accuracy: 0.8813791275024414


In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, History
import tensorflow as tf
from tensorflow.keras.regularizers import l2

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(dskb['Article'], dskb['label'], test_size=0.2, random_state=42)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)  # Encode training labels
y_test = label_encoder.transform(y_test)  # Encode test labels

# Text preprocessing
max_words = 10000
max_len = 500

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Build LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                   output_dim=embedding_dim,weights=[embedding_matrix]))
model.add(LSTM(64, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=l2(0.01)))
model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01)))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [47]:
# Train model
history = History()
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

model.fit(X_train_pad, y_train, epochs=5, batch_size=128, validation_split=0.2, callbacks=[history, early_stopping])

# Predict
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype("int32")

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Epoch 1/5
503/503 ━━━━━━━━━━━━━━━━━━━━ 395s 779ms/step - accuracy: 0.8870 - loss: 0.7077 - val_accuracy: 0.9989 - val_loss: 0.0561
Epoch 2/5
503/503 ━━━━━━━━━━━━━━━━━━━━ 397s 789ms/step - accuracy: 0.9993 - loss: 0.0545 - val_accuracy: 0.9996 - val_loss: 0.0417
Epoch 3/5
503/503 ━━━━━━━━━━━━━━━━━━━━ 398s 791ms/step - accuracy: 0.9996 - loss: 0.0419 - val_accuracy: 0.9996 - val_loss: 0.0363
Epoch 4/5
503/503 ━━━━━━━━━━━━━━━━━━━━ 401s 798ms/step - accuracy: 0.9997 - loss: 0.0365 - val_accuracy: 0.9996 - val_loss: 0.0327
Epoch 5/5
503/503 ━━━━━━━━━━━━━━━━━━━━ 413s 822ms/step - accuracy: 0.9999 - loss: 0.0327 - val_accuracy: 0.9996 - val_loss: 0.0320
Restoring model weights from the end of the best epoch: 5.
628/628 ━━━━━━━━━━━━━━━━━━━━ 50s 79ms/step
Accuracy: 0.9997012696041823
Precision: 0.9997988129966804
Recall: 0.9995977069294981
F1 Score: 0.9996982498491249


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# Pad sequences
X = pad_sequences(tokenizer.texts_to_sequences(dskb['Article']), maxlen=100)
y = dskb['label']

# Train-test split
from sklearn.model_selection import train_test_split
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

# Ensure labels are NumPy arrays and of the correct type
y_train = np.array(y_train, dtype=np.float32)
y_val = np.array(y_val, dtype=np.float32)


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train model
history = model.fit(X_train_pad, y_train, epochs=5, batch_size=128, validation_split=0.2, callbacks=[history, early_stopping])

# Evaluate model
val_loss, val_accuracy = model.evaluate(X_test, y_test)
print(f"Validation Accuracy: {val_accuracy}")

In [29]:
import optuna
import numpy as np
import gc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Bidirectional, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

#tokenizer = Tokenizer()
# Pad sequences
X = pad_sequences(tokenizer.texts_to_sequences(dskb['Article']), maxlen=100)
y = dskb['label']

# Train-test split
from sklearn.model_selection import train_test_split
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Ensure labels are NumPy arrays and of the correct type
y_train = np.array(y_train, dtype=np.float32)
y_val = np.array(y_val, dtype=np.float32)

embedding_dim = 100  # Fixed dimension matching your embedding matrix

# Define the objective function for Optuna
def create_model(trial):
    model = Sequential()
    
    # Hyperparameters to optimize
    embedding_dim = trial.suggest_categorical('embedding_dim', [50, 100, 200])
    #conv_filters = trial.suggest_int('conv_filters', 32, 128)
    #kernel_size = trial.suggest_int('kernel_size', 2, 5)
    lstm_units = trial.suggest_int('lstm_units', 50, 256)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    #lstm_units = trial.suggest_int('lstm_units', 50, 256)
    #dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    # Dynamically adjust embedding_matrix
    adjusted_embedding_matrix = embedding_matrix[:, :embedding_dim]
    
    
    model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=embedding_dim,
                    #weights=[adjusted_embedding_matrix],  # Use the pretrained weights
                    trainable=False))  # Trainable embeddings
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.3)),
    #model.add(Dense(32, activation='relu'))
    #model.add(Dropout(0.2))
    
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def objective(trial):
    model = create_model(trial)
    
    # Train the model
    history = model.fit(X_train, y_train,
                        epochs=3,
                        batch_size=64,
                        validation_data=(X_val, y_val),
                        verbose=0)
    # Perform garbage collection
    gc.collect()
    
    # Get the validation accuracy
    val_accuracy = history.history['val_accuracy'][-1]
    return val_accuracy

# Create a study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)
# Run Optuna study
#study = optuna.create_study(direction="minimize")  # Minimize (1 - accuracy) to maximize accuracy

# Best hyperparameters and accuracy
print("Best hyperparameters:", study.best_params)
print("Best trial validation accuracy:", 1 - study.best_value)
# Print validation accuracy of the best trial
best_trial = study.best_trial
print("Best validation accuracy:", best_trial.value)


[I 2025-01-13 15:14:24,305] A new study created in memory with name: no-name-572981f3-a8e5-4297-8c9e-0427f1790af7
[I 2025-01-13 15:18:08,928] Trial 0 finished with value: 1.0 and parameters: {'embedding_dim': 200, 'lstm_units': 216, 'dropout_rate': 0.4533826774578452}. Best is trial 0 with value: 1.0.
[I 2025-01-13 15:22:02,689] Trial 1 finished with value: 1.0 and parameters: {'embedding_dim': 200, 'lstm_units': 146, 'dropout_rate': 0.3172318983033062}. Best is trial 0 with value: 1.0.
[I 2025-01-13 15:25:26,725] Trial 2 finished with value: 0.9993153810501099 and parameters: {'embedding_dim': 100, 'lstm_units': 94, 'dropout_rate': 0.46179398945445316}. Best is trial 0 with value: 1.0.
[I 2025-01-13 15:29:05,047] Trial 3 finished with value: 0.9993153810501099 and parameters: {'embedding_dim': 100, 'lstm_units': 247, 'dropout_rate': 0.4626752397524019}. Best is trial 0 with value: 1.0.
[I 2025-01-13 15:32:49,929] Trial 4 finished with value: 1.0 and parameters: {'embedding_dim': 200, 

Best hyperparameters: {'embedding_dim': 200, 'lstm_units': 216, 'dropout_rate': 0.4533826774578452}
Best trial validation accuracy: 0.0
Best validation accuracy: 1.0


In [33]:
import optuna.visualization as vis


import matplotlib.pyplot as plt

# Plot optimization history (accuracy over trials)
optuna.visualization.plot_optimization_history(study)
plt.show()

# Plot parameter importances (which hyperparameters affected accuracy most)
optuna.visualization.plot_param_importances(study)
plt.show()
